# Technique 8: Redis-Backed Memory

## Overview

This technique stores conversation history in Redis, allowing for persistent storage across sessions and distributed systems. Memory is shared across multiple instances.

## Pros
- Persistent storage (survives restarts)
- Distributed (can share memory across instances)
- Fast access
- Good for production systems
- Uses modern LangChain v1.0+ patterns (no deprecation warnings)

## Cons
- Requires Redis server
- Additional infrastructure to manage
- Network latency (if Redis is remote)

## Use Case
Production applications that need persistent memory across sessions or distributed systems with multiple instances.

## Implementation

The code below demonstrates this technique:

In [ ]:
"""Technique 8: Redis-Backed Memory (LCEL Pattern)===============================================This technique stores conversation history in Redis, allowing for persistentstorage across sessions and distributed systems. Memory is shared acrossmultiple instances. Uses modern LCEL pattern.Pros:- Persistent storage (survives restarts)- Distributed (can share memory across instances)- Fast access- Good for production systems- Uses modern LangChain v1.0+ patterns (no deprecation warnings)Cons:- Requires Redis server- Additional infrastructure to manage- Network latency (if Redis is remote)Use Case: Production applications that need persistent memory acrosssessions or distributed systems with multiple instances."""from langchain_openai import ChatOpenAIfrom langchain_core.chat_history import BaseChatMessageHistoryfrom langchain_community.chat_message_histories import RedisChatMessageHistoryfrom langchain_core.runnables.history import RunnableWithMessageHistoryfrom langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholderfrom dotenv import load_dotenvimport osimport sysfrom typing import Dict# Add parent directory to path for utilsimport pathlibsys.path.append(str(pathlib.Path().absolute().parent))from utils.token_counter import (    count_tokens,     count_messages_tokens,    print_token_stats,    print_token_summary)load_dotenv()# Store for Redis chat message historiesstore: Dict[str, BaseChatMessageHistory] = {}def get_session_history(session_id: str) -> BaseChatMessageHistory:    """Get or create Redis chat message history for a session."""    if session_id not in store:        redis_url = os.getenv("REDIS_URL", "redis://localhost:6379/0")        store[session_id] = RedisChatMessageHistory(            url=redis_url,            ttl=600,  # Time to live in seconds (10 minutes)            session_id=session_id        )    return store[session_id]def create_redis_memory_agent():    """Create an agent with Redis-backed memory using LCEL pattern."""        # Initialize the LLM    llm = ChatOpenAI(        model="gpt-4o",        temperature=0.7,        openai_api_key=os.getenv("OPENAI_API_KEY")    )        # Create a prompt template with message history placeholder    prompt = ChatPromptTemplate.from_messages([        ("system", "You are a helpful AI assistant. Have a natural conversation with the user."),        MessagesPlaceholder(variable_name="history"),        ("human", "{input}")    ])        # Create the chain using LCEL    chain = prompt | llm        # Wrap with message history (Redis-backed)    chain_with_history = RunnableWithMessageHistory(        chain,        get_session_history,        input_messages_key="input",        history_messages_key="history",    )        return chain_with_historydef demonstrate_redis_memory():    """Demonstrate Redis memory using LCEL pattern."""    print("=" * 60)    print("Technique 8: Redis-Backed Memory (LCEL Pattern)")    print("=" * 60)    print("Note: Requires Redis server running")    print("Using default Redis URL: redis://localhost:6379/0")    print("Using modern LangChain v1.0+ patterns with RunnableWithMessageHistory")    print()        session_id = "demo_redis_session"    config = {"configurable": {"session_id": session_id}}        try:        chain = create_redis_memory_agent()    except Exception as e:        print(f"Error connecting to Redis: {e}")        print("Please ensure Redis is running or set REDIS_URL environment variable")        return        # Simulate a conversation    conversations = [        "Hi, I'm Grace",        "I'm a product manager",        "I work on mobile apps",        "What's my name?",        "What do I do?"    ]        total_input_tokens = 0    total_output_tokens = 0        for i, user_input in enumerate(conversations, 1):        print(f"User: {user_input}")                # Count input tokens (user message + history)        input_tokens = count_tokens(user_input)        history = get_session_history(session_id)        if history.messages:            input_tokens += count_messages_tokens(history.messages)        total_input_tokens += input_tokens                response = chain.invoke(            {"input": user_input},            config=config        )        print(f"Agent: {response.content}")                # Count output tokens        output_tokens = count_tokens(response.content)        total_output_tokens += output_tokens                # Count current memory tokens        history = get_session_history(session_id)        memory_tokens = count_messages_tokens(history.messages) if history.messages else 0                print_token_stats(input_tokens, output_tokens, memory_tokens)        print()        # Show the stored memory    print("\n" + "-" * 60)    print("Stored Memory (in Redis):")    print("-" * 60)    history = get_session_history(session_id)    for message in history.messages:        print(f"{message.type.capitalize()}: {message.content}")    print()        # Show total token usage    final_memory = count_messages_tokens(history.messages) if history.messages else 0    print_token_summary(        total_input_tokens,         total_output_tokens,         final_memory    )

## Example Usage

Run the demonstration function to see the technique in action:

In [ ]:
# Run the demonstration function
demonstrate_redis_memory()
